# Analisis

In [ ]:
import numpy as np
from sklearn import linear_model
from matplotlib import pyplot as plt

In [ ]:
#-----------------------------------
## Parámetros globales de la antena
#-----------------------------------

Fs = 23.328e9 #Frecuencia de muestreo [Hz] 
Ts = 1/Fs
BW = 1.5e9 
corte = 150
Nfft= 1024 # nro de frecs -> conviene que sea multiplo de 2 para calcular la FFT de manera eficiente. 
#Tiene que ser mayor a la longitud de la señal (285 para 2m, 439 para 3m)

#------------------------------
#Elegir frecuencia portadora
#------------------------------
portadora = input('portadora? (Fc7 o Fc8)')

if portadora == 'Fc7':
    fc = 7.290e9 
    Fc = '7.29GHz'
elif portadora == 'Fc8':
    fc = 8.748e9
    Fc = '8.74GHz'
else:
    print('Error: portadora debe ser Fc7 o Fc8')

## Importo variables

In [3]:
import pickle
if portadora == 'Fc8':
    with open('mayo_Fc8.pkl','rb') as f:
        Mayo8 = pickle.load(f)
    with open('junio_Fc8.pkl','rb') as g:
        Junio8 = pickle.load(g)
    with open('julio_Fc8.pkl','rb') as r:
        Julio8 = pickle.load(r)
    with open('agosto_Fc8.pkl','rb') as t:
        Agosto8 = pickle.load(t)

    print(Mayo8.keys())
    print(Junio8.keys())
    print(Julio8.keys())
    print(Agosto8.keys())
else:
    with open('mayo_Fc7.pkl','rb') as f:
        Mayo7 = pickle.load(f)
    with open('junio_Fc7.pkl','rb') as g:
        Junio7 = pickle.load(g)
    with open('julio_Fc7.pkl','rb') as r:
        Julio7 = pickle.load(r)
    with open('agosto_Fc7.pkl','rb') as t:
        Agosto7 = pickle.load(t)
    print(Mayo7.keys())
    print(Junio7.keys())
    print(Julio7.keys())
    print(Agosto7.keys())

In [5]:
#Las mediciones ya transformadas y los datos de altura y angulo de incidencia por fecha


dict_keys(['h1_may', 'h2_may', 'f_may', 'S1_may', 'S2_may'])
dict_keys(['h1_jun', 'h2_jun', 'f_jun', 'S1_jun', 'S2_jun', 'hz_jun', 'Z2_jun', 'tita_jun'])
dict_keys(['h1_jul', 'h2_jul', 'f_jul', 'S1_jul', 'S2_jul', 'hz_jul', 'Z2_jul', 'tita_jul'])
dict_keys(['h_ag', 'f_ag', 'S1A_ag', 'S1B_ag', 'S2_ag', 'M2_ag', 'hz_ag', 'Z2_ag', 'MZ2_ag', 'tita_ag'])


In [4]:
#chequeo que las frecuencias sean iguales
# fm = Mayo8.get('f_may')
# fjn = Junio8.get('f_jun')
# fjl = Julio8.get('f_jul')
# fa = Agosto8.get('f_ag')

# L = [fm,fjn,fjl,fa]
# out = (np.diff(np.vstack(L).reshape(len(L),-1),axis=0)==0).all()
# print('todas las frecuencias son siempre iguales?',out)

# if out:
#     f = fm
#     print('renombro las frecuencias como f')
# else:
#     print('diferentes')

f = Mayo8.get('f_may')

todas las frecuencias son siempre iguales? True
renombro las frecuencias como f


## 
